# KerasNLP 시작하기

**저자:** [Jonathan Bischof](https://github.com/jbischof)  
**생성일:** 2022/12/15  
**최종편집일:** 2023/07/01  
**설명:** KerasNLP API에 대한 소개입니다.

## 소개

KerasNLP는 전체 개발 주기를 지원하는 자연어 처리 라이브러리입니다.
우리의 워크플로우는 모듈형 구성 요소로 이루어져 있으며,
최첨단 사전 트레이닝된 가중치와 아키텍처를 제공하며,
사용자 필요에 따라 쉽게 커스터마이즈할 수 있습니다.

이 라이브러리는 코어 Keras API의 확장입니다.
모든 높은 레벨 모듈은 [`Layers`](https://codecompose7.github.io/keras-doc-kr.github.io/api/layers/) 또는 [`Models`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/)입니다.
Keras에 익숙하다면, 축하합니다! 이미 대부분의 KerasNLP를 이해하고 있는 것입니다.

KerasNLP는 Keras 3를 사용하여, TensorFlow, Pytorch, Jax와 함께 작동합니다.
아래 가이드에서는, 모델 트레이닝을 위해 `jax` 백엔드를 사용하고,
입력 전처리를 효율적으로 처리하기 위해, [tf.data](https://www.tensorflow.org/guide/data)를 사용합니다.
하지만, 자유롭게 다른 것을 사용해도 됩니다!
이 가이드는 백엔드를 TensorFlow 또는 PyTorch로 바꿔도 아무런 변경 없이 실행됩니다.
아래 `KERAS_BACKEND`를 업데이트하기만 하면 됩니다.

이 가이드는 감정 분석 예제를 통해, 모듈식 접근 방식을 여섯 가지 복잡도 레벨에서 보여줍니다:

-   사전 트레이닝된 분류기를 사용한 추론
-   사전 트레이닝된 백본을 미세 조정
-   사용자 제어 전처리로 미세 조정
-   커스텀 모델을 미세 조정
-   백본 모델을 사전 트레이닝
-   처음부터 직접 트랜스포머 모델 빌드 및 트레이닝

가이드 전체에서, 우리는 Keras 공식 마스코트인 Keras 교수(Professor Keras)를 시각적 참조로 사용하여 자료의 복잡성을 설명합니다:

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_evolution.png" alt="drawing" height="250"/>

In [1]:
# 이 노트북은 Keras cv가 설치되어 있다는 가정 하에 진행됩니다.
# 이 노트북은 Keras 3이 설치되어 있다는 가정 하에 진행됩니다.
#
# !pip install -q --upgrade keras-cv
# !pip install -q --upgrade keras  # Upgrade to Keras 3.

In [2]:
# import os

# os.environ["KERAS_BACKEND"] = "tensorflow"
# os.environ["KERAS_BACKEND"] = "jax"
# os.environ["KERAS_BACKEND"] = "torch"

In [3]:
from keras import backend

print(backend.backend())

tensorflow


In [4]:
import keras

print(keras.__version__)

3.4.1


In [5]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # 또는 "tensorflow", "torch"

import keras_nlp
import keras

# 이 가이드에서 모든 트레이닝을 가속화하기 위해 혼합 정밀도 사용.
keras.mixed_precision.set_global_policy("mixed_float16")

## API 빠른 시작

가장 높은 레벨의 API는 `keras_nlp.models`입니다.
이 심볼들은 문자열을 토큰으로, 토큰을 밀집 특성으로,
그리고 dense 특성을 작업별 출력으로 변환하는 전체 과정을 다룹니다.
각 `XX` 아키텍처(예: `Bert`)에 대해, 다음 모듈을 제공합니다:

-   **Tokenizer**: `keras_nlp.models.XXTokenizer`
    -   **기능**: 문자열을 토큰 ID 시퀀스로 변환합니다.
    -   **중요성**: 문자열의 raw 바이트는 유용한 특성으로 사용되기엔 차원이 너무 높으므로,
        먼저 작은 수의 토큰으로 매핑합니다.
        예를 들어, `"The quick brown fox"`는 `["the", "qu", "##ick", "br", "##own", "fox"]`로 변환됩니다.
    -   **상속받는 클래스**: [`keras.layers.Layer`](https://codecompose7.github.io/keras-doc-kr.github.io/api/layers/base_layer#layer-class).
  
-   **Preprocessor**: `keras_nlp.models.XXPreprocessor`
    -   **기능**: 문자열을 토큰화로 시작하여, 백본이 사용할 수 있는 전처리된, 텐서 딕셔너리로 변환합니다.
    -   **중요성**: 각 모델은 입력을 이해하기 위해, 구분 기호 토큰과 같은 특수 토큰 및 추가 텐서를 사용합니다.
        예를 들어 입력 세그먼트를 구분하거나, 패딩 토큰을 식별하는 기능이 포함됩니다.
        모든 시퀀스를 동일한 길이로 패딩하면, 계산 효율성이 높아집니다.
    -   **구성 요소**: `XXTokenizer`.
    -   **상속받는 클래스**: [`keras.layers.Layer`](https://codecompose7.github.io/keras-doc-kr.github.io/api/layers/base_layer#layer-class).
  
-   **Backbone**: `keras_nlp.models.XXBackbone`
    -   **기능**: 전처리된 텐서를 dense 특성으로 변환합니다. *문자열 처리는 하지 않으므로, 먼저 전처리기를 호출해야 합니다.*
    -   **중요성**: 백본은 입력 토큰을 dense 특성으로 압축하여, 후속 작업에 사용할 수 있도록 합니다.
        백본 모델은 일반적으로 대량의 비지도 학습 데이터를 사용하여, 언어 모델링 작업으로 사전 트레이닝된 것입니다.
        이러한 정보를 새로운 작업에 전이하는 것은 현대 NLP에서 중요한 돌파구입니다.
    -   **상속받는 클래스**: [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class).
  
-   **Task**: 예를 들어, `keras_nlp.models.XXClassifier`
    -   **기능**: 문자열을 작업별 출력(예: 분류 확률)으로 변환합니다.
    -   **중요성**: 작업 모델은 문자열 전처리와 백본 모델을 작업별 `Layers`와 결합하여,
        문장 분류, 토큰 분류, 텍스트 생성 등의 문제를 해결합니다.
        추가된 `Layers`는 라벨이 지정된 데이터를 사용하여 미세 조정해야 합니다.
    -   **구성 요소**: `XXBackbone`과 `XXPreprocessor`.
    -   **상속받는 클래스**: [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class).

다음은 `BertClassifier`의 모듈 계층 구조입니다(모든 관계는 구성적 관계입니다):

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/class_diagram.png" alt="drawing" height="300"/>

모든 모듈은 독립적으로 사용할 수 있으며, **사전 설정**된 아키텍처와 가중치로 클래스를 인스턴스화하는,
`from_preset()` 메서드를 갖고 있습니다. (아래 예 참조)

## 데이터

우리는 IMDB 영화 리뷰의 감정 분석 예시를 사용합니다.
이 작업에서는 텍스트를 사용하여 리뷰가 긍정적(`label = 1`)인지 부정적(`label = 0`)인지를 예측합니다.

데이터는 [`keras.utils.text_dataset_from_directory`](https://codecompose7.github.io/keras-doc-kr.github.io/api/data_loading/text#textdatasetfromdirectory-function)를 사용하여 로드되며,
이는 강력한 [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) 형식을 이용합니다.

In [6]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# 비지도 학습 예제 제거
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   850k      0  0:01:36  0:01:36 --:--:--  897k


In [7]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# 첫 번째 리뷰 확인
# 형식은 (리뷰 텍스트 텐서, 라벨 텐서)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'I remember watching this in the 1970s - then I have just recently borrowed a couple of episodes from our public library.<br /><br />With a nearly 30 year hiatus, I have come to another conclusion. Most of the principals interviewed in this series - some at the center of power like Traudl Junge (Hitler\'s Secretary),Karl Doenitz (head of Germany\'s navy) Anthony Eden (UK) - are long gone but their first hand accounts will live on.From Generals and Admirals to Sergeants, Russian civilians, concentration camp survivors, all are on record here. <br /><br />I can remember the Lord Mountbatten interview (killed in the 1970s) <br /><br />This is truly a gem and I believe the producer of this series was knighted by Queen Elizabeth for this work - well deserved.<br /><br />Seeing these few episodes from the library makes me want to buy the set.<br /><br />This is the 

## 사전 트레이닝된 분류기로 추론하기

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_beginner.png" height="250" alt="drawing" />

KerasNLP에서 가장 높은 레벨의 모듈은 **태스크**입니다.
**태스크**는 (일반적으로 사전 트레이닝된) **백본** 모델과 태스크 특화 레이어들로 구성된 [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class)입니다.
다음은 [`keras_nlp.models.BertClassifier`](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/models/bert/bert_classifier#bertclassifier-class)를 사용하는 예시입니다.

**참고**: 출력은 클래스별 로짓입니다.
(예: `[0, 0]`은 긍정일 확률이 50%임을 나타냅니다)
출력은 이진 분류의 경우 \[negative, positive\]입니다.

In [8]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# 참고: 배치 입력이 필요하므로, 문자열을 iterable로 래핑해야 합니다.
classifier.predict(["I love modular workflows in keras-nlp!"])

100%|██████████| 454/454 [00:00<00:00, 601kB/s]
100%|██████████| 2.04k/2.04k [00:00<00:00, 2.48MB/s]
100%|██████████| 226k/226k [00:00<00:00, 284kB/s]
100%|██████████| 50.3M/50.3M [00:04<00:00, 11.5MB/s]
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
100%|██████████| 16.8M/16.8M [00:02<00:00, 5.96MB/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[-1.289,  1.395]], dtype=float16)

모든 **태스크**에는 사전 설정된 전처리, 아키텍처 및 가중치로 [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class) 인스턴스를 생성하는 `from_preset` 메서드가 있습니다.
이는 우리가 [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class)에서 허용하는 모든 형식의 raw 문자열을 전달하고, 태스크에 맞는 출력 값을 받을 수 있음을 의미합니다.

이 특정 **프리셋**은 `"bert_tiny_uncased_en"` **백본**을 `sst2`로 파인 튜닝한 모델로,
이는 Rotten Tomatoes 영화 리뷰 감성 분석을 수행한 모델입니다.
데모에서는 `tiny` 아키텍처를 사용하지만, 더 큰 모델을 사용하면 최신 성능(SoTA)을 얻을 수 있습니다.
`BertClassifier`에 사용 가능한 모든 태스크별 프리셋은,
[models 페이지](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/models/)에서 확인할 수 있습니다.

이제 IMDB 데이터셋에서 분류기를 평가해 봅시다.
여기서 [`keras.Model.compile`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model_training_apis#compile-method)를 호출할 필요는 없습니다.
`BertClassifier`와 같은 모든 **태스크** 모델은 기본적으로 컴파일된 상태로 제공되므로,
[`keras.Model.evaluate`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model_training_apis#evaluate-method)를 바로 호출할 수 있습니다.
물론, 원한다면 새로운 메트릭을 추가하기 위해, 일반적인 컴파일 방식을 사용할 수 있습니다.

출력 값은 \[loss, accuracy\]입니다.

In [9]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.4521 - sparse_categorical_accuracy: 0.7908


[0.45500361919403076, 0.7903199791908264]

결과는 78%의 정확도로, 아무런 트레이닝 없이 이 정도 성능을 얻었습니다. 나쁘지 않네요!

## 사전 트레이닝된 BERT 백본 파인 튜닝

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_intermediate.png" height="250" alt="drawing" />

태스크에 맞는 라벨링된 텍스트가 있으면, 커스텀 분류기를 파인 튜닝하여 성능을 향상시킬 수 있습니다.
IMDB 리뷰 감성을 예측하려면, Rotten Tomatoes 데이터보다 IMDB 데이터를 사용하는 것이 더 나은 성능을 낼 것입니다.
또한, 많은 태스크에서 관련 사전 트레이닝된 모델이 없을 수도 있습니다. (예: 고객 리뷰 분류(categorizing))

파인 튜닝의 워크플로우는 위와 거의 동일하지만, 전체 분류기 대신 **백본** 전용 **프리셋**을 요청하는 차이만 있습니다.
**백본** **프리셋**이 전달되면, **태스크** `Model`은 태스크에 맞는 레이어들을 무작위로 초기화하고 트레이닝 준비를 합니다.
`BertClassifier`에 사용 가능한 모든 **백본** 프리셋은,
[models 페이지](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/models/)에서 확인할 수 있습니다.

분류기를 트레이닝하려면, 다른 [`keras.Model`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model#model-class)처럼 [`keras.Model.fit`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model_training_apis#fit-method)를 사용하면 됩니다.
위에서와 마찬가지로 **태스크**에 대해, [`keras.Model.compile`](https://codecompose7.github.io/keras-doc-kr.github.io/api/models/model_training_apis#compile-method)을 스킵하고, 컴파일 기본값을 사용할 수 있습니다.
전처리가 포함되어 있으므로, raw 데이터를 바로 전달할 수 있습니다.

In [10]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

100%|██████████| 507/507 [00:00<00:00, 222kB/s]
100%|██████████| 16.8M/16.8M [00:03<00:00, 4.66MB/s]
100%|██████████| 547/547 [00:00<00:00, 1.09MB/s]
100%|██████████| 226k/226k [00:00<00:00, 268kB/s]


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - loss: 0.5245 - sparse_categorical_accuracy: 0.7212 - val_loss: 0.3178 - val_sparse_categorical_accuracy: 0.8633


여기서는 한 번의 트레이닝만으로 검증 정확도가 0.78에서 0.87로 크게 상승하는 것을 볼 수 있습니다.
IMDB 데이터셋이 `sst2`보다 훨씬 작음에도 불구하고 말이죠.

## 사용자 제어 전처리로 파인 튜닝

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_advanced.png" height="250" alt="drawing" />

일부 고급 트레이닝 시나리오에서는, 사용자가 전처리를 직접 제어하기를 원할 수 있습니다.
대규모 데이터셋의 경우, 예제를 미리 전처리하여 디스크에 저장하거나,
[`tf.data.experimental.service`](https://www.tensorflow.org/api_docs/python/tf/data/experimental/service)를 사용하여 별도의 작업자 풀에서 전처리할 수 있습니다.
또는, 입력을 다루기 위해 커스텀 전처리가 필요한 경우도 있습니다.

**태스크** `Model` 생성자에 `preprocessor=None`을 전달하여 자동 전처리를 건너뛰거나,
대신 커스텀 `BertPreprocessor`를 전달할 수 있습니다.

### 동일한 프리셋에서 전처리 분리하기

각 모델 아키텍처에는 자체 `from_preset` 생성자가 있는, 병렬 **전처리** `Layer`가 있습니다.
이 `Layer`에 대해 동일한 **프리셋**을 사용하면, **태스크**와 일치하는 **전처리**를 반환합니다.

이 워크플로우에서는 `tf.data.Dataset.cache()`를 사용하여,
fit 시작 전에 전처리를 한 번만 계산하고 그 결과를 캐시한 다음,
3번의 에포크 동안 모델을 트레이닝합니다.

**참고:** [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data)를 사용하여,
Jax 또는 PyTorch 백엔드에서 전처리를 실행할 수 있습니다.
입력 데이터셋은 트레이닝 중에 백엔드 네이티브 텐서 타입으로 자동 변환됩니다.
실제로 [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data)의 전처리 효율성을 고려할 때,
모든 백엔드에서 이를 사용하는 것이 좋은 관행입니다.

In [11]:
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

# 전처리를 `map()`을 사용해, 트레이닝 및 테스트 데이터의 각 샘플에 적용합니다.
# 성능을 조정하려면 [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE)과
# `prefetch()` 옵션을 사용할 수 있습니다.
# 성능 세부 사항은 https://www.tensorflow.org/guide/data_performance에서 확인하세요.

# 참고: `cache()`는 트레이닝 데이터가 CPU 메모리에 맞는 경우에만 호출하세요!
imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 12ms/step - loss: 0.5329 - sparse_categorical_accuracy: 0.7144 - val_loss: 0.3163 - val_sparse_categorical_accuracy: 0.8637
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.2889 - sparse_categorical_accuracy: 0.8802 - val_loss: 0.3161 - val_sparse_categorical_accuracy: 0.8687
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.2156 - sparse_categorical_accuracy: 0.9143 - val_loss: 0.3374 - val_sparse_categorical_accuracy: 0.8682


세 번의 에포크 후, 우리의 검증 정확도는 0.88로 증가했습니다.
이는 데이터셋의 크기와 모델의 크기에 의해 결정됩니다.
90% 이상의 정확도를 달성하려면, `"bert_base_en_uncased"`와 같은 더 큰 **프리셋**을 시도해 보세요.
사용 가능한 모든 **백본** 프리셋은,
keras.io [모델 페이지](https://codecompose7.github.io/keras-doc-kr.github.io/api/keras_nlp/models/)에서 확인할 수 있습니다.

### 커스텀 전처리

커스텀 전처리가 필요한 경우, raw 문자열을 토큰으로 매핑하는 `Tokenizer` 클래스를 직접 사용할 수 있습니다.
이 클래스에는 사전 트레이닝과 일치하는 어휘를 얻기 위한, `from_preset()` 생성자가 있습니다.

**참고:** `BertTokenizer`는 기본적으로 시퀀스를 패딩하지 않으므로, 출력은 길이가 가변적인 형식으로 나타납니다.
아래의 `MultiSegmentPacker`는 이러한 가변 길이 시퀀스를 dense 텐서 타입([`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) 또는 `torch.Tensor`)으로 패딩 처리합니다.

In [12]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")
tokenizer(["I love modular workflows!", "Libraries over frameworks!"])

# 직접 패커를 작성하거나 `Layers` 중 하나를 사용할 수 있습니다.
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    # 참고: 이 값은 프리셋의 `sequence_length`보다 길 수 없으며,
    # 커스텀 전처리기에는 이를 확인하는 과정이 없습니다!
    sequence_length=64,
)

# 이 함수는 텍스트 샘플 `x`와 해당 레이블 `y`를 입력받아,
# 텍스트를 BERT 모델에 적합한 형식으로 변환합니다.
def preprocessor(x, y):
    token_ids, segment_ids = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
    return x, y


imdb_train_preprocessed = imdb_train.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
imdb_test_preprocessed = imdb_test.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

# 전처리된 예시 출력
print(imdb_train_preprocessed.unbatch().take(1).get_single_element())

({'token_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  2023,  2034,  2048,  3692,  1997,  2023,  4038,  2186,
        2020,  2200,  4326,  1998,  2027,  4694,  1005,  1056,  2200,
        6057,  1998,  2018,  1037,  3689,  5783,  2073,  3021,  1006,
        1996,  2388,  1007,  2001,  8084,  2007,  2035,  1996,  5156,
        3471,  1999,  2166,  2021,  2008,  5783,  2001,  1037,  2978,
        2139, 24128,  1998,  2134,  1005,  1056,  4666,  2092,  2007,
       16215,  4038,  3787,  2029,  2003,  2763,  2339,  2009,  2001,
         102], dtype=int32)>, 'segment_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(64,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  T

## 커스텀 모델을 사용한 파인 튜닝

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_advanced.png" height="250" alt="drawing" />

더 고급 응용 프로그램의 경우, 적절한 **태스크** `Model`이 제공되지 않을 수 있습니다.
이 경우, 커스텀 `Layer`와 함께 사용할 수 있는 **백본** `Model`에 직접 액세스할 수 있으며,
이는 자체 `from_preset` 생성자를 가지고 있습니다.
자세한 예시는 [전이 학습 가이드](https://codecompose7.github.io/keras-doc-kr.github.io/guides/transfer_learning/)에서 확인할 수 있습니다.

**백본** `Model`은 자동 전처리를 포함하지 않지만,
이전 워크플로우에서 보여준 것처럼,
동일한 **프리셋**을 사용하는 일치하는 **전처리기**와 함께 사용할 수 있습니다.

이번 워크플로우에서는, 백본 모델을 동결하고, 새로운 입력에 맞게, 두 개의 트레이닝 가능한 트랜스포머 레이어를 추가하여 실험합니다.

**참고**: 우리는 BERT의 시퀀스 출력을 사용하고 있으므로, `pooled_dense` 레이어에 대한 경고를 무시할 수 있습니다.

In [13]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

imdb_train_preprocessed = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_preprocessed = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = keras_nlp.layers.TransformerEncoder(
        num_heads=2,
        intermediate_dim=512,
        dropout=0.1,
    )(sequence)
# [CLS] 토큰 출력을 사용하여 분류
outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)
model.summary()
model.fit(
    imdb_train_preprocessed,
    validation_data=imdb_test_preprocessed,
    epochs=3,
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │        198,272 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_1     │ (None, None, 128)      │        198,272 │ transformer_encoder[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_4 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │            258 │ get_item_4[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,782,722 (18.24 MB)

 Trainable params: 396,802 (1.51 MB)

 Non-trainable params: 4,385,920 (16.73 MB)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - loss: 0.6489 - sparse_categorical_accuracy: 0.6518 - val_loss: 0.5828 - val_sparse_categorical_accuracy: 0.7006
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.5081 - sparse_categorical_accuracy: 0.7533 - val_loss: 0.4922 - val_sparse_categorical_accuracy: 0.7708
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.4537 - sparse_categorical_accuracy: 0.7844 - val_loss: 0.4574 - val_sparse_categorical_accuracy: 0.7904


이 모델은 `BertClassifier` 모델에 비해 트레이닝 가능한 파라미터 수가 10%밖에 되지 않지만,
상당히 좋은 정확도를 달성합니다.
캐시된 전처리를 감안해도, 각 트레이닝 단계가 약 1/3의 시간을 소요합니다.

## 백본 모델 사전 트레이닝

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_expert.png" height="250" alt="drawing" />

당신의 도메인에서, 대규모 라벨이 없는 데이터셋에 접근할 수 있나요?
이러한 데이터셋의 크기가 BERT, RoBERTa 또는 GPT2와 같은 유명한 백본 모델을 트레이닝시키는 데 사용된 데이터와 유사한 크기(XX+ GiB)인가요?
그렇다면, 도메인에 특화된 사전 트레이닝을 통해 자체 백본 모델을 학습하는 것이 도움이 될 수 있습니다.

NLP 모델은 일반적으로 언어 모델링 작업을 통해 사전 트레이닝되며,
이는 입력 문장에 보이는 단어를 기준으로 가려진 단어를 예측하는 방식입니다.
예를 들어, `"The fox [MASK] over the [MASK] dog"`라는 입력에서,
모델은 `["jumped", "lazy"]`를 예측해야 합니다.
이 모델의 하위 레이어는 **백본**으로 패키징되어, 새로운 작업과 관련된 레이어와 결합됩니다.

KerasNLP 라이브러리는 SoTA **백본**과 **토크나이저**를 프리셋 없이 처음부터 트레이닝할 수 있도록 지원합니다.

이번 워크플로우에서는, IMDB 리뷰 텍스트를 사용하여 BERT **백본**을 사전 학습합니다.
데이터 처리 복잡성을 줄이기 위해, "next sentence prediction" (NSP) 손실을 생략하였으며,
이는 RoBERTa와 같은 이후 모델에서도 제외되었습니다.
자세한 내용은 원본 논문을 복제하는 단계별 가이드를 제공하는,
[Transformer 사전 트레이닝](https://codecompose7.github.io/keras-doc-kr.github.io/guides/keras_nlp/transformer_pretraining/#pretraining)을 참조하세요.

### 전처리

In [14]:
# 모든 BERT `en` 모델들은 동일한 어휘집을 사용하므로,
# "bert_tiny_en_uncased"의 전처리기를 재사용합니다.
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=256,
)
packer = preprocessor.packer
tokenizer = preprocessor.tokenizer

# 일부 입력 토큰을 "[MASK]" 토큰으로 교체하는 keras.Layer
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=0.25,
    mask_selection_length=64,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
    unselectable_token_ids=[
        tokenizer.token_to_id(x) for x in ["[CLS]", "[PAD]", "[SEP]"]
    ],
)


def preprocess(inputs, label):
    inputs = preprocessor(inputs)
    masked_inputs = masker(inputs["token_ids"])
    # 마스킹 레이어 출력을 (features, labels, weights)로 분리하여
    # keras.Model.fit()에서 사용할 수 있도록 합니다.
    features = {
        "token_ids": masked_inputs["token_ids"],
        "segment_ids": inputs["segment_ids"],
        "padding_mask": inputs["padding_mask"],
        "mask_positions": masked_inputs["mask_positions"],
    }
    labels = masked_inputs["mask_ids"]
    weights = masked_inputs["mask_weights"]
    return features, labels, weights


pretrain_ds = imdb_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
pretrain_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# ID 103은 "masked"된 토큰입니다.
print(pretrain_ds.unbatch().take(1).get_single_element())

({'token_ids': <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([  101,  3532,  2097,  2052,  2022,  5291,  2058,  1999,  2010,
        6542,   103,  2002,  2071,  2023,   103,  7570,   103,  7028,
        2446,  1011,  2694, 15581,   103,  2239,  1997,  2010,  4438,
        2377,  1012,  2009,  1005,   103,   103,   103,  2200,  2210,
         103,  2001,  2985,  2006,  2009,   103,  1037,  2754,  4125,
        2099,  1010,  1037,  4937,   103,  1998,   103, 18154,  2872,
         103,  3413,  2125,   103,   103,   103,   103,  1996,  3185,
        2001,  1999,   103,   103,   103,  2046,  2394,  1010,  2007,
         103,  2394,  2376,  5889,  5681, 12954,  9709,  2037,  3210,
        1012,   103,  2878,  2537,  2018,  2019,   103,  2601,  1998,
       25596, 14644,  2100,  2514,   103,  2009,  1012,  1998,  2074,
        2073,  2001, 15489,  2121, 10024,   103,  1999,   103,  3185,
        4312,  1029,  5796,   103,   103,  2243,  2435,  2023,  3374,
       20969,  1996,  3949,  

### 사전 트레이닝 모델

In [19]:
# BERT 백본 모델 생성
backbone = keras_nlp.models.BertBackbone(
    vocabulary_size=tokenizer.vocabulary_size(),
    num_layers=2,
    num_heads=2,
    hidden_dim=128,
    intermediate_dim=512,
)

# 언어 모델링 헤드 생성
mlm_head = keras_nlp.layers.MaskedLMHead(
    token_embedding=backbone.token_embedding,
)

# inputs = {
#     "token_ids": keras.Input(shape=(None,), dtype=tf.int32, name="token_ids"),
#     "segment_ids": keras.Input(shape=(None,), dtype=tf.int32, name="segment_ids"),
#     "padding_mask": keras.Input(shape=(None,), dtype=tf.int32, name="padding_mask"),
#     "mask_positions": keras.Input(shape=(None,), dtype=tf.int32, name="mask_positions"),
# }
inputs = [
    keras.Input(shape=(None,), dtype=tf.int32, name="token_ids"),
    keras.Input(shape=(None,), dtype=tf.int32, name="segment_ids"),
    keras.Input(shape=(None,), dtype=tf.int32, name="padding_mask"),
]

# 인코딩된 토큰 시퀀스
sequence = backbone(inputs)["sequence_output"]

# 각 마스킹된 입력 토큰에 대해 출력 단어 예측.
# 입력 토큰 임베딩을 사용해 인코딩된 벡터에서 어휘 로짓으로 프로젝션합니다.
# 이는 트레이닝 효율성을 향상시키는 것으로 알려져 있습니다.
# outputs = mlm_head(sequence, mask_positions=inputs["mask_positions"])
outputs = mlm_head(sequence, mask_positions=keras.Input(shape=(None,), dtype=tf.int32, name="mask_positions"))

# 사전 트레이닝 모델 정의 및 컴파일
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.summary()
pretraining_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(learning_rate=5e-4),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)

# IMDB 데이터셋으로 사전 트레이닝 진행
pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=3,  # 더 높은 정확도를 위해 6으로 증가 가능
)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone_4           │ [(None, 128), (None,   │      4,385,920 │ token_ids[0][0],       │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ padding_mask[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mask_positions            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masked_lm_head_4          │ (None, None, 30522)    │      3,954,106 │ bert_backbone_4[0][1], │
│ (MaskedLMHead)            │                        │                │ mask_positions[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,433,210 (16.91 MB)

 Trainable params: 4,433,210 (16.91 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3


ValueError: Exception encountered when calling Softmax.call().

[1mDimensions must be equal, but are 256 and 64 for '{{node functional_2_1/bert_backbone_4_1/transformer_layer_0_1/self_attention_layer_1/softmax_18_1/add}} = AddV2[T=DT_HALF](functional_2_1/bert_backbone_4_1/transformer_layer_0_1/self_attention_layer_1/transpose_2, functional_2_1/bert_backbone_4_1/transformer_layer_0_1/self_attention_layer_1/softmax_18_1/mul)' with input shapes: [?,2,256,256], [?,1,1,64].[0m

Arguments received by Softmax.call():
  • inputs=tf.Tensor(shape=(None, 2, 256, 256), dtype=float16)
  • mask=tf.Tensor(shape=(None, 1, 1, 64), dtype=int32)

사전 트레이닝 후, `backbone` 서브모델을 저장하여 새로운 작업에 사용하세요!

## 처음부터 직접 트랜스포머 빌드 및 트레이닝

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_expert.png" height="250" alt="drawing" />

새로운 트랜스포머 아키텍처를 구현하고 싶으신가요?
KerasNLP 라이브러리는 SoTA(최첨단) 아키텍처를 구축하는 데 사용되는,
모든 낮은 레벨 모듈을 `models` API에 제공합니다.
여기에는 `keras_nlp.tokenizers` API가 포함되어 있어,
`WordPieceTokenizer`, `BytePairTokenizer`, 또는 `SentencePieceTokenizer`를 사용하여,
직접 서브워드 토크나이저를 트레이닝할 수 있습니다.

이 워크플로우에서는, IMDB 데이터에 대해 커스텀 토크나이저를 트레이닝하고,
커스텀 트랜스포머 아키텍처로 백본을 설계합니다.
간단하게 하기 위해, 바로 분류 작업에 대해 트레이닝을 진행합니다.
더 자세한 내용이 궁금하신가요?
커스텀 트랜스포머를 프리트레이닝하고 파인튜닝하는 전체 가이드를,
[keras.io](https://codecompose7.github.io/keras-doc-kr.github.io/guides/keras_nlp/transformer_pretraining/)에서 작성했습니다.

### IMDB 데이터에서 커스텀 어휘 트레이닝

In [20]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    imdb_train.map(lambda x, y: x),
    vocabulary_size=20_000,
    lowercase=True,
    strip_accents=True,
    reserved_tokens=["[PAD]", "[START]", "[END]", "[MASK]", "[UNK]"],
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=True,
    strip_accents=True,
    oov_token="[UNK]",
)

### 커스텀 토크나이저로 데이터 전처리

In [21]:
packer = keras_nlp.layers.StartEndPacker(
    start_value=tokenizer.token_to_id("[START]"),
    end_value=tokenizer.token_to_id("[END]"),
    pad_value=tokenizer.token_to_id("[PAD]"),
    sequence_length=512,
)


def preprocess(x, y):
    token_ids = packer(tokenizer(x))
    return token_ids, y


imdb_preproc_train_ds = imdb_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
imdb_preproc_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(imdb_preproc_train_ds.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([    1,   141,   116,   146,   190,   105,  4794,     7,   451,
        7718,     7,  3026,    20,  7712,    17,   116,   146, 19109,
        4794,   327,    19,   114,   156,   116,   279,   102,    36,
          52,   215,   118,   265,    44,   272,   134,   102,  3499,
         155,  2915,   178,    19,    52,  1537,   141,   116,   233,
         178,   105,   112,   461,  5807,  7761,  1513,   116,   174,
         123,  7176,    13,    44,  1161,  1397,    12,    62,  3182,
          19,    14,   114,   111,   155,   373,    52,    12,    47,
         133,    44,   211,   236,   147,    17,   147,  1216,    17,
         147,   824,    19,   147,    19,    52,   326,   133,    52,
          12,    56,  7264,   111,    50, 12198,  1767,    37,    99,
        2179,   101,    99,  1369,    19,    99,  9646,   286,    99,
        7103,    19,   114,   119,   137,   482,    12,    63,  1627,
          19,     2,     0,     0,     0,  

### 작은 트랜스포머 설계

In [22]:
token_id_input = keras.Input(
    shape=(None,),
    dtype="int32",
    name="token_ids",
)
outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(vocab),
    sequence_length=packer.sequence_length,
    embedding_dim=64,
)(token_id_input)
outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=2,
    intermediate_dim=128,
    dropout=0.1,
)(outputs)
# "[START]" 토큰을 사용하여 분류
outputs = keras.layers.Dense(2)(outputs[:, 0, :])
model = keras.Model(
    inputs=token_id_input,
    outputs=outputs,
)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)               │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 64)            │       1,260,544 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_2                │ (None, None, 64)            │          33,472 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item_10 (GetItem)                │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,294,146 (4.94 MB)

 Trainable params: 1,294,146 (4.94 MB)

 Non-trainable params: 0 (0.00 B)

### 트랜스포머를 직접 분류 목표(objective)에 맞춰 트레이닝

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)
model.fit(
    imdb_preproc_train_ds,
    validation_data=imdb_preproc_val_ds,
    epochs=3,
)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 0.7218 - sparse_categorical_accuracy: 0.5222 - val_loss: 0.5311 - val_sparse_categorical_accuracy: 0.7583
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.4649 - sparse_categorical_accuracy: 0.7785 - val_loss: 0.4911 - val_sparse_categorical_accuracy: 0.7610
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3220 - sparse_categorical_accuracy: 0.8685

흥미롭게도, 우리가 설계한 커스텀 분류기는 `"bert_tiny_en_uncased"`를 미세 조정한 성능과 비슷합니다!
90% 이상의 정확도를 달성하고,
사전 트레이닝의 장점을 보기 위해서는 `"bert_base_en_uncased"`와 같은,
더 큰 **프리셋**을 사용해야 합니다.
